In [1]:
import sys
import os


sys.path.append(os.path.abspath("../..")) #make the folder "automl" part of this

RESULTS_PATH = 'results.csv'
OPTUNA_DATABASE = 'study_results.db'


In [2]:
from automl.loggers.result_logger import ResultLogger
import optuna
import optuna.visualization as vis
from automl.utils.optuna_utils import load_study_from_database


c:\rgoncalo\ricardo-goncalo-thesis-project\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load the experiment

In [ ]:
experiment_path = 'data\\experiments\\HyperparameterOptimizationPipeline_1'

# Evaluation of HyperparameterOptimizationPipeline

In [4]:
hyperparameter_optimization_results : ResultLogger = ResultLogger(input={
                                        "logger_directory" : experiment_path,
                                        "filename" : RESULTS_PATH
                                      })

hyperparameter_optimization_results.proccess_input()

Opening a log in directory: data\experiments\HyperparameterOptimizationPipeline_0, with name:
Type of object with name: <class 'automl.loggers.result_logger.ResultLogger'> and name passed: 
ResultLogger: Created logger in directory: data\experiments\HyperparameterOptimizationPipeline_0
ResultLogger: Results dataframe with filename results.csv already existed with columns Index(['experiment', 'num_episodes', 'discount_factor', 'epsilon_start',
       'epsilon_end', 'epsilon_decay', 'hidden_layers', 'hidden_size',
       'target_update_rate', 'learning_rate', 'memory_capacity', 'result'],
      dtype='object')


In [5]:
optuna_study = load_study_from_database(database_path=hyperparameter_optimization_results.lg.logDir + '\\' + OPTUNA_DATABASE)

KeyError: 'Record does not exist.'

In [ ]:
hyperparameter_optimization_results.plot_bar_graph(x_axis='experiment', y_axis='result')

In [ ]:
fig = vis.plot_param_importances(optuna_study)
fig.show()

# Global evaluation of configurations

In [ ]:

results_of_configurations : dict[str, ResultLogger] = {}

for configuration_name in os.listdir(experiment_path):
    configuration_path = os.path.join(experiment_path, configuration_name)
    
    if os.path.isdir(configuration_path):  # Ensure it's a file, not a subdirectory
        
        results_of_configurations[configuration_name] = ResultLogger(input={
                                        "logger_directory" : configuration_path,
                                        "filename" : RESULTS_PATH
                                      })
        results_of_configurations[configuration_name].proccess_input()


In [ ]:
print(f"Configurations:  {results_of_configurations.keys()}")

# Global view of performance

In [ ]:
import matplotlib.pyplot as plt


for name, results_logger in results_of_configurations.items():

    #results_logger.plot_graph(x_axis='episode', y_axis=[('total_reward', name)], to_show=False)
    results_logger.plot_confidence_interval(x_axis='episode', y_column='total_reward', show_std=False, to_show=False, y_values_label=name)
    
plt.show()

## Worst Performances

# Smaller study

In [ ]:
configurations_to_study = ["configuration_20"]

In [ ]:


for configuration_name in configurations_to_study:
    
    results_logger = results_of_configurations[configuration_name]

    #results_logger.plot_graph(x_axis='episode', y_axis=[('total_reward', name)], to_show=False)
    results_logger.plot_confidence_interval(x_axis='episode', y_column='total_reward',show_std=False, to_show=False, y_values_label=configuration_name)
    results_logger.plot_linear_regression(x_axis='episode', y_axis='total_reward', to_show=False, y_label=configuration_name + '_linear')
    
plt.show()

In [ ]:
agents_to_study : dict[str, ResultLogger]= {}

for configuration_name in configurations_to_study:
    
    results_logger = results_of_configurations[configuration_name]
    
    for agent_name in ["agent_1", "agent_2"]:
      
        agent_results_logger = ResultLogger(input={
                                            "logger_directory" : f"{results_logger.lg.logDir}\\{agent_name}",
                                            "filename" : RESULTS_PATH
                                          })

        agents_to_study[f"{configuration_name}_{agent_name}"] = agent_results_logger
        
        agent_results_logger.proccess_input()


In [ ]:
for agent_name, agent_results_logger in agents_to_study.items():
    
    #results_logger.plot_graph(x_axis='episode', y_axis=[('total_reward', name)], to_show=False)
    agent_results_logger.plot_confidence_interval(x_axis='episode', y_column='total_reward',show_std=False, to_show=False, y_values_label=agent_name)
    
plt.show()